# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.




## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx" "${HOME}/Projects/gonb" "${HOME}/Projects/gopjrt" "${HOME}/Projects/bsplines"
%goworkfix

	- Added replace rule for module "github.com/janpfeifer/gonb" to local directory "/home/janpf/Projects/gonb".
	- Added replace rule for module "github.com/gomlx/gopjrt" to local directory "/home/janpf/Projects/gopjrt".
	- Added replace rule for module "github.com/gomlx/bsplines" to local directory "/home/janpf/Projects/bsplines".
	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".


## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [21]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/ml/data"
    "github.com/janpfeifer/must"

    _ "github.com/gomlx/gomlx/backends/xla"
)

var (
	flagDataDir    = flag.String("data", "~/tmp/imdb", "Directory to cache downloaded and generated dataset files.")
	flagEval       = flag.Bool("eval", true, "Whether to evaluate the model on the validation data in the end.")
	flagVerbosity  = flag.Int("verbosity", 1, "Level of verbosity, the higher the more verbose.")
	flagCheckpoint = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
)

func AssertDownloaded() {
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    if !data.FileExists(*flagDataDir) {
        must.M(os.MkdirAll(*flagDataDir, 0777))
    }
    must.M(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [7]:
import "github.com/gomlx/gomlx/examples/imdb"

%%
AssertDownloaded()
imdb.PrintSample(3)


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
┌────────────────────────────────────────────────────────────┐
│                                                            │
│    [Sample 0 - label 0]                                    │
│    <START> nothing i dislike more than a kung fu movie     │
│    that plays for laughs it is the main reason i can t     │
│    stand jackie chan or his lookalikes he was not          │
│    always a clown i must add my young auntie is            │
│    slapstick martial arts of the worst kind it is a        │
│    perfect example of how the subgenre was brought down    │
│    to the mud by endless silly antics and childish         │
│    behavior unless you are 5 year old i really don t       │
│    understand how anyone could find this kind of film      │
│    funny but humor is indeed a very subjective thing       │
│    personally i think thi

## Training

We will create 3 different types of models for this demo: **Bag of Words** (**"bow"**), **Convolutionals** (**"cnn"**) and **Transformers** (**"transformer"**).

### Model Configuration

As with other demos we leverage the `context.Context` object to store all model and training parameters. 
One can set specific parameters using the `-set` command line flag.

The [`imdb.CreateDefaultContext()`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/train.go) method sets all the default values for the hyperparameters that may be used by any of the 3 model types. The parameter "model" specify the model type.

In [16]:
import (
    "golang.org/x/exp/maps"
    "github.com/gomlx/gomlx/ml/context"
)

// settings is bound to a "-set" flag to be used to set context hyperparameters.
var settings = commandline.CreateContextSettingsFlag(imdb.CreateDefaultContext(), "set")

// ContextFromSettings is the default context (createDefaultContext) changed by -set flag.
func ContextFromSettings() *context.Context {
    ctx := imdb.CreateDefaultContext()
    must.M(commandline.ParseContextSettings(ctx, *settings))
    return ctx
}

%% -set="model=cnn"
fmt.Printf("Model types: %q\n", maps.Keys(imdb.ValidModels))
ctx := ContextFromSettings()
fmt.Println(commandline.SprintContextSettings(ctx))

Model types: ["bow" "cnn" "transformer"]
Context hyperparameters:
	"activation": (string) 
	"adam_dtype": (string) 
	"adam_epsilon": (float64) 1e-07
	"batch_size": (int) 32
	"cnn_dropout_rate": (float64) 0.5
	"cnn_normalization": (string) 
	"cnn_num_layers": (float64) 5
	"cosine_schedule_steps": (int) 0
	"dropout_rate": (float64) 0.1
	"eval_batch_size": (int) 200
	"fnn_dropout_rate": (float64) 0.3
	"fnn_normalization": (string) 
	"fnn_num_hidden_layers": (int) 2
	"fnn_num_hidden_nodes": (int) 32
	"fnn_residual": (bool) true
	"imdb_content_max_len": (int) 200
	"imdb_include_separators": (bool) false
	"imdb_mask_word_task_weight": (float64) 0
	"imdb_max_vocab": (int) 20000
	"imdb_token_embedding_size": (int) 32
	"imdb_use_unsupervised": (bool) false
	"imdb_word_dropout_rate": (float64) 0
	"l1_regularization": (float64) 0
	"l2_regularization": (float64) 0
	"learning_rate": (float64) 0.0001
	"model": (string) cnn
	"normalization": (string) batch
	"num_checkpoints": (int) 3
	"optimizer": (s

### Bag Of Words Model (bow)

This is the simplest model we are going to train: it embeds each token of the sentence (default size of the is 32 numbers) sum them up, and pass that through a FNN.

The [code in `imdb.BagOfWordsModelGraph`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/model_bagofwords.go) looks like this:

```go
// BagOfWordsModelGraph builds the computation graph for the "bag of words" model: simply the sum of the embeddings
// for each token included.
func BagOfWordsModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
	embed, _ := EmbedTokensGraph(ctx, inputs[0])

	// Take the max over the content length, and put an FNN on top.
	// Shape transformation: [batch_size, content_len, embed_size] -> [batch_size, embed_size]
	embed = ReduceMax(embed, 1)
	logits := fnn.New(ctx, embed, 1).Done()
	return []*Node{logits}
}
```

We played a bit with the hyperparameters to get to ~85% accuracy on the validation data.

The [code for `imdb.TrainModel` is here](https://github.com/gomlx/gomlx/blob/main/examples/imdb/train.go).
It's a straight forward GoMLX training loop.

In [27]:
%% --set="model=bow;l2_regularization=1e-3;learning_rate=1e-4;normalization=none;train_steps=10000"
ctx := ContextFromSettings()
imdb.TrainModel(ctx, *flagDataDir, *flagCheckpoint, *flagEval, *flagVerbosity)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Backend "xla":	xla:cuda - PJRT "cuda" plugin (/usr/local/lib/gomlx/pjrt/pjrt_c_api_cuda_plugin.so) v0.54
Model: bow


Training (10000 steps):    7% [=>......................................] (137 steps/s) [5s:1m7s] [step=719] [loss+=0.690] [~loss+=0.692] [~loss=0.692] [~acc=56.79%]         

Training (10000 steps):  100% [========================================] (157 steps/s) [step=9999] [loss+=0.403] [~loss+=0.341] [~loss=0.325] [~acc=86.22%]         


Metric: accuracy

Metric: loss

	[Step 10000] median train step: 5465 microseconds

Results on train-eval:
	Mean Loss+Regularization (#loss+): 0.262
	Mean Loss (#loss): 0.246
	Mean Accuracy (#acc): 90.88%
Results on test-eval:
	Mean Loss+Regularization (#loss+): 0.362
	Mean Loss (#loss): 0.347
	Mean Accuracy (#acc): 84.80%


### Convolution Model (cnn)

The function [`imdb.CnnModelGraph`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/model_cnn.go) creates a 1D convolution model, with arbitrary number of convolutions. After the convolution, it behaves the same way as the Bag Of Words model.

The core of the convolution model looks like this:

```go
	// 1D Convolution: embed is [batch_size, content_len, embed_size].
	numConvolutions := context.GetParamOr(ctx, "cnn_num_layers", 5)
	logits := embed
	for convIdx := range numConvolutions {
		ctx := ctx.Inf("%03d_conv", convIdx)
		residual := logits
		if convIdx > 0 {
			logits = NormalizeSequence(ctx, logits)
		}
		logits = layers.Convolution(ctx, embed).KernelSize(7).Filters(embedSize).Strides(1).Done()
		logits = activations.ApplyFromContext(ctx, logits)
		if dropoutNode != nil {
			logits = layers.Dropout(ctx, logits, dropoutNode)
		}
		if residual.Shape().Equal(logits.Shape()) {
			logits = Add(logits, residual)
		}
	}

	// Take the max over the content length, and put an FNN on top.
	// Shape transformation: [batch_size, content_len, embed_size] -> [batch_size, embed_size]
	logits = ReduceMax(logits, 1)
	logits = fnn.New(ctx, logits, 1).Done()
	logits.AssertDims(batchSize, 1)
```

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [32]:
%% --set="model=cnn;l2_regularization=1e-3;learning_rate=1e-4;normalization=layer;train_steps=10000"
ctx := ContextFromSettings()
imdb.TrainModel(ctx, *flagDataDir, *flagCheckpoint, *flagEval, *flagVerbosity)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Backend "xla":	xla:cuda - PJRT "cuda" plugin (/usr/local/lib/gomlx/pjrt/pjrt_c_api_cuda_plugin.so) v0.54
Model: cnn


Training (10000 steps):    7% [=>......................................] (115 steps/s) [8s:1m20s] [step=719] [loss+=0.718] [~loss+=0.717] [~loss=0.694] [~acc=51.62%]        

Training (10000 steps):  100% [========================================] (140 steps/s) [step=9999] [loss+=0.153] [~loss+=0.096] [~loss=0.077] [~acc=97.62%]          


Metric: accuracy

Metric: loss

	[Step 10000] median train step: 5230 microseconds

Results on train-eval:
	Mean Loss+Regularization (#loss+): 0.077
	Mean Loss (#loss): 0.058
	Mean Accuracy (#acc): 98.36%
Results on test-eval:
	Mean Loss+Regularization (#loss+): 0.776
	Mean Loss (#loss): 0.757
	Mean Accuracy (#acc): 82.12%


### Transformer Model

Finally a Transformer version of the model, as defined in the ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) famous paper. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data, but that is beyond the scope of this small test.

The code is in [`imdb.TransformerModelGraph`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/model_transformer.go), and the core of it looks like this:

```go
    ...
	// Add the requested number of attention layers.
	numAttLayers := context.GetParamOr(ctx, "transformer_num_att_layers", 1)
	numAttHeads := context.GetParamOr(ctx, "transformer_num_att_heads", 2)
	attKeySize := context.GetParamOr(ctx, "transformer_att_key_size", 8)
	for layerNum := range numAttLayers {
		// Each layer in its own scope.
		ctx := ctx.Inf("%03d_attention_layer", layerNum)
		residual := embed
		embed = layers.MultiHeadAttention(ctx.In("000_attention"), embed, embed, embed, numAttHeads, attKeySize).
			SetKeyMask(mask).SetQueryMask(mask).
			SetOutputDim(embedSize).
			SetValueHeadDim(embedSize).Done()
		if dropoutNode != nil {
			embed = layers.Dropout(ctx.In("001_dropout"), embed, dropoutNode)
		}
		embed = NormalizeSequence(ctx.In("002_normalization"), embed)
		attentionOutput := embed

		// Transformers recipe: 2 dense layers after attention.
		embed = fnn.New(ctx.In("003_fnn"), embed, embedSize).NumHiddenLayers(1, embedSize).Done()
		if dropoutNode != nil {
			embed = layers.Dropout(ctx.In("004_dropout"), embed, dropoutNode)
		}
		embed = Add(embed, attentionOutput)
		embed = NormalizeSequence(ctx.In("005_normalization"), embed)

		// Residual connection:
		if layerNum > 0 {
			embed = Add(residual, embed)
		}
	}
    ...
```

With only 5000 steps we got ~87% on the test data -- and significant overfitting as well.

In [38]:
%% --set="model=transformer;normalization=none;activation=swish;l2_regularization=1e-3;cnn_dropout_rate=0.5;fnn_dropout_rate=0.3;learning_rate=1e-4;train_steps=5000"
ctx := ContextFromSettings()
imdb.TrainModel(ctx, *flagDataDir, *flagCheckpoint, *flagEval, *flagVerbosity)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Backend "xla":	xla:cuda - PJRT "cuda" plugin (/usr/local/lib/gomlx/pjrt/pjrt_c_api_cuda_plugin.so) v0.54
Model: transformer


Training (5000 steps):   14% [====>...................................] (58 steps/s) [16s:1m14s] [step=724] [loss+=0.689] [~loss+=0.693] [~loss=0.692] [~acc=52.24%]        

Training (5000 steps):  100% [========================================] (56 steps/s) [step=4999] [loss+=0.182] [~loss+=0.226] [~loss=0.190] [~acc=93.32%]          


Metric: accuracy

Metric: loss

	[Step 5000] median train step: 16066 microseconds

Results on train-eval:
	Mean Loss+Regularization (#loss+): 0.200
	Mean Loss (#loss): 0.163
	Mean Accuracy (#acc): 94.27%
Results on test-eval:
	Mean Loss+Regularization (#loss+): 0.345
	Mean Loss (#loss): 0.309
	Mean Accuracy (#acc): 87.34%
